In [0]:
!pip install pandas

In [0]:
!wget https://raw.githubusercontent.com/kmsaumcis/mcis6273_f19_datamining/master/homework/data/openpowerlifting-2019-10-24.csv

In [0]:
import pandas as pd

In [0]:
df = pd.read_csv("openpowerlifting-2019-10-24.csv", low_memory=False)

In [0]:
df.Date = pd.to_datetime(df.Date, errors='coerce')

In [0]:
df = df.query('Date>=1999')

In [85]:
df

,Name,Sex,Event,Equipment,Age,AgeClass,BirthYearClass,Division,BodyweightKg,WeightClassKg,Squat1Kg,Squat2Kg,Squat3Kg,Squat4Kg,Best3SquatKg,Bench1Kg,Bench2Kg,Bench3Kg,Bench4Kg,Best3BenchKg,Deadlift1Kg,Deadlift2Kg,Deadlift3Kg,Deadlift4Kg,Best3DeadliftKg,TotalKg,Place,Wilks,McCulloch,Glossbrenner,IPFPoints,Tested,Country,Federation,Date,MeetCountry,MeetState,MeetName
0,Abbie Murphy,F,SBD,Wraps,29.0,24-34,NaN,F-OR,59.80,60,80.0,92.5,105.0,NaN,105.00,45.0,50.0,55.0,NaN,55.00,110.0,120.0,130.0,NaN,130.00,290.00,4,324.16,324.16,286.42,511.15,NaN,NaN,GPC-AUS,2018-10-27,Australia,VIC,Melbourne Cup
1,Abbie Tuong,F,SBD,Wraps,29.0,24-34,NaN,F-OR,58.50,60,100.0,110.0,120.0,NaN,120.00,55.0,62.5,67.5,NaN,67.50,130.0,140.0,145.0,NaN,145.00,332.50,2,378.07,378.07,334.16,595.65,NaN,NaN,GPC-AUS,2018-10-27,Australia,VIC,Melbourne Cup
2,Ainslee Hooper,F,B,Raw,40.0,40-44,40-49,F-OR,55.40,56,NaN,NaN,NaN,NaN,NaN,27.5,32.5,-35.0,NaN,32.50,NaN,NaN,NaN,NaN,NaN,32.50,1,38.56,38.56,34.12,313.97,NaN,NaN,GPC-AUS,2018-10-27,Australia,VIC,Melbourne Cup
3,Amy Moldenhauer,F,SBD,Wraps,23.0,20-23,NaN,F-OR,60.00,60,-105.0,-105.0,105.0,NaN,105.00,67.5,72.5,-75.0,NaN,72.50,132.5,-140.0,-140.0,NaN,132.50,310.00,3,345.61,345.61,305.37,547.04,NaN,NaN,GPC-AUS,2018-10-27,Australia,VIC,Melbourne Cup
4,Andrea Rowan,F,SBD,Wraps,45.0,45-49,40-49,F-OR,104.00,110,120.0,130.0,140.0,NaN,140.00,70.0,75.0,80.0,NaN,80.00,150.0,160.0,170.0,NaN,170.00,390.00,3,321.25,338.91,274.56,550.08,NaN,NaN,GPC-AUS,2018-10-27,Australia,VIC,Melbourne Cup
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
489504,Jack Delmar,M,SBD,Single-ply,NaN,NaN,NaN,Boys,70.90,74.8,NaN,NaN,NaN,NaN,124.74,NaN,NaN,NaN,NaN,74.84,NaN,NaN,NaN,NaN,140.61,340.19,6,252.48,252.48,244.54,344.13,Yes,NaN,THSPA,2011-02-10,USA,TX,Needville
489505,Andrew Kubena,M,SBD,Single-ply,NaN,NaN,NaN,Boys,71.58,74.8,NaN,NaN,NaN,NaN,111.13,NaN,NaN,NaN,NaN,65.77,NaN,NaN,NaN,NaN,142.88,319.78,7,235.64,235.64,228.14,325.33,Yes,NaN,THSPA,2011-02-10,USA,TX,Needville
489506,Barrett Janecek,M,SBD,Single-ply,NaN,NaN,NaN,Boys,72.44,74.8,NaN,NaN,NaN,NaN,83.91,NaN,NaN,NaN,NaN,54.43,NaN,NaN,NaN,NaN,102.06,240.40,8,175.60,175.60,169.92,258.32,Yes,NaN,THSPA,2011-02-10,USA,TX,Needville
489507,Greg Sexton,M,SBD,Single-ply,NaN,NaN,NaN,Boys,80.60,82.1,NaN,NaN,NaN,NaN,213.19,NaN,NaN,NaN,NaN,115.67,NaN,NaN,NaN,NaN,231.33,560.19,1,380.65,380.65,366.65,485.98,Yes,NaN,THSPA,2011-02-10,USA,TX,Needville


In [0]:
features = [
     'Sex',
     'Age',
     'BodyweightKg',
     'Best3SquatKg',
      'Best3BenchKg',
     'Best3DeadliftKg',
     'TotalKg'
]

In [0]:
df = df.dropna(subset=features)

In [0]:
df = df[features]

In [89]:
df

,Sex,Age,BodyweightKg,Best3SquatKg,Best3BenchKg,Best3DeadliftKg,TotalKg
0,F,29.0,59.80,105.00,55.00,130.00,290.00
1,F,29.0,58.50,120.00,67.50,145.00,332.50
3,F,23.0,60.00,105.00,72.50,132.50,310.00
4,F,45.0,104.00,140.00,80.00,170.00,390.00
5,F,37.0,74.00,142.50,82.50,145.00,370.00
...,...,...,...,...,...,...,...
489457,M,13.5,112.94,210.92,133.81,183.70,528.44
489460,M,15.5,120.79,208.65,97.52,195.04,501.22
489468,M,13.5,114.31,147.42,111.13,145.15,403.70
489475,M,16.5,136.67,215.46,124.74,195.04,535.24


In [0]:
df = pd.get_dummies(df, dtype=float)

In [91]:
df

,Age,BodyweightKg,Best3SquatKg,Best3BenchKg,Best3DeadliftKg,TotalKg,Sex_F,Sex_M
0,29.0,59.80,105.00,55.00,130.00,290.00,1.0,0.0
1,29.0,58.50,120.00,67.50,145.00,332.50,1.0,0.0
3,23.0,60.00,105.00,72.50,132.50,310.00,1.0,0.0
4,45.0,104.00,140.00,80.00,170.00,390.00,1.0,0.0
5,37.0,74.00,142.50,82.50,145.00,370.00,1.0,0.0
...,...,...,...,...,...,...,...,...
489457,13.5,112.94,210.92,133.81,183.70,528.44,0.0,1.0
489460,15.5,120.79,208.65,97.52,195.04,501.22,0.0,1.0
489468,13.5,114.31,147.42,111.13,145.15,403.70,0.0,1.0
489475,16.5,136.67,215.46,124.74,195.04,535.24,0.0,1.0


clearly how many features are now in your dataframe? 8

In [93]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
Sum_of_squared_distances = [] 
K = range(2, 15)
for k in K:
      km = KMeans(n_clusters=k, n_init=20)
      km = km.fit(df)
      Sum_of_squared_distances.append(km.inertia_)
      silh_score = silhouette_score(df, km.labels_)
      print("k = {} | silhouette_score = {}".format(k, silh_score))

k = 2 | silhouette_score = 0.544566638320242
k = 3 | silhouette_score = 0.5002869574312617
k = 4 | silhouette_score = 0.45978431683370613
k = 5 | silhouette_score = 0.42715218421214196
k = 6 | silhouette_score = 0.398512319631528
k = 7 | silhouette_score = 0.3760810878065334
k = 8 | silhouette_score = 0.35836654176840066
k = 9 | silhouette_score = 0.3351603116025923
k = 10 | silhouette_score = 0.3155755804657863
k = 11 | silhouette_score = 0.29995811202763606
k = 12 | silhouette_score = 0.28476369944554514
k = 13 | silhouette_score = 0.2683520627614267
k = 14 | silhouette_score = 0.2527297119126668


What is the optimal k according the silhouette score? 2

What else is interesting about the scores? 
Silhouette score seems to decrease with increasing k.

In [97]:
optimal_k = 2
km = KMeans(n_clusters=optimal_k, n_init=150)
km = km.fit(df)

for i in range(0, optimal_k):
  l = list(zip(df.columns, km.cluster_centers_[i]))
  l.sort(key=lambda x: x[1], reverse=True)

  print('CLUSTER : {}\n'.format(i))
  for attr, val in l[:]:
    print('\t{} : {}\n'.format(attr, val))

CLUSTER : 0

	TotalKg : 363.0671464535957

	Best3DeadliftKg : 151.84975377613767

	Best3SquatKg : 130.05524313221656

	Best3BenchKg : 81.16278155641166

	BodyweightKg : 71.14394554367905

	Age : 28.876854959809148

	Sex_M : 0.5167167211375394

	Sex_F : 0.4832832788624606

CLUSTER : 1

	TotalKg : 647.6756324621472

	Best3DeadliftKg : 249.20592483716473

	Best3SquatKg : 239.91472820186246

	Best3BenchKg : 158.5558307706391

	BodyweightKg : 96.33688721056339

	Age : 29.15461061866841

	Sex_M : 0.9880155571633784

	Sex_F : 0.011984442836621673

